In [14]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, HashingVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
import itertools
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import warnings
warnings.filterwarnings("ignore")
from gensim.models import Word2Vec
from sklearn.linear_model import PassiveAggressiveClassifier,LogisticRegression
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [15]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [16]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [17]:
df = df.dropna()
df.reset_index(inplace = True)

In [18]:
df

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [19]:
X = df.drop('label',axis =1)
Y = df['label']

In [20]:
ps = PorterStemmer()
corpus = []
words = []
for i in range(0,len(df)):
    review = re.sub('[^a-zA-Z0-9]',' ',df['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    statements = ' '.join(review)
    corpus.append(statements)
    words.append(review)

In [21]:
count = CountVectorizer(max_features=5000,ngram_range=(1,3))
X_count = count.fit_transform(corpus).toarray()
tfidf = TfidfVectorizer(max_features=5000,ngram_range=(1,3))
X_tfidf = tfidf.fit_transform(corpus).toarray()


In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 '15 civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay 2 5 million releas everyon arrest dakota access pipelin',
 'fbi clos

In [23]:
X_count.shape

(18285, 5000)

In [24]:
Y =df["label"]

In [25]:
X_train_count,X_test_count,Y_train_count,Y_test_count = train_test_split(X_count,Y,test_size=0.33,random_state = 27)
X_train_tfidf,X_test_tfidf,Y_train_tfidf,Y_test_tfidf = train_test_split(X_tfidf,Y,test_size=0.33,random_state = 27)


In [27]:
count.get_feature_names_out()[:10]

array(['000', '000 email', '000 job', '00pm', '00pm water',
       '00pm water cooler', '01', '10', '10 000', '10 year'], dtype=object)

In [28]:
count.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 5000,
 'min_df': 1,
 'ngram_range': (1, 3),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [30]:
df_count = pd.DataFrame(X_train_count, columns=tfidf.get_feature_names_out())
df_count.head()

,000,000 email,000 job,00pm,00pm water,00pm water cooler,01,10,10 000,10 year,...,zealand,zero,zika,zika viru,zionist,zone,zone new,zone new york,zu,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
df_tfidf = pd.DataFrame(X_train_tfidf, columns=tfidf.get_feature_names_out())
df_tfidf.head()


,000,000 email,000 job,00pm,00pm water,00pm water cooler,01,10,10 000,10 year,...,zealand,zero,zika,zika viru,zionist,zone,zone new,zone new york,zu,zuckerberg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
classfier_count = MultinomialNB(alpha=0.1)
classfier_tfidf = MultinomialNB(alpha=0.1)

In [34]:
classfier_count.fit(X_train_count,Y_train_count)
Y_pred_count = classfier_count.predict(X_test_count)
classfier_tfidf.fit(X_train_tfidf,Y_train_tfidf)
Y_pred_tfidf = classfier_tfidf.predict(X_test_tfidf)

In [35]:
acc_count = accuracy_score(Y_test_count,Y_pred_count)
classification_count = classification_report(Y_test_count,Y_pred_count)
confusion_matrix_count = confusion_matrix(Y_test_count,Y_pred_count)
acc_tfidf = accuracy_score(Y_test_tfidf,Y_pred_tfidf)
classification_tfidf = classification_report(Y_test_tfidf,Y_pred_tfidf)
confusion_matrix_tfidf = confusion_matrix(Y_test_tfidf,Y_pred_tfidf)

In [36]:
print("FOR COUNT VECTORIZER: \n")
print(" \n Accuracy : ",acc_count,"\n","Classification report \n",classification_count,"\n","Confusion matrix \n",confusion_matrix_count,"\n\n\n")
print("FOR TFIDF VECTORIZER: \n")
print(" \n Accuracy : ",acc_tfidf,"\n","Classification report \n",classification_tfidf,"\n","Confusion matrix \n",confusion_matrix_tfidf)

FOR COUNT VECTORIZER: 

 
 Accuracy :  0.9042253521126761 
 Classification report 
               precision    recall  f1-score   support

           0       0.91      0.92      0.92      3390
           1       0.90      0.89      0.89      2645

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035
 
 Confusion matrix 
 [[3116  274]
 [ 304 2341]] 



FOR TFIDF VECTORIZER: 

 
 Accuracy :  0.8783761391880696 
 Classification report 
               precision    recall  f1-score   support

           0       0.85      0.95      0.90      3390
           1       0.92      0.79      0.85      2645

    accuracy                           0.88      6035
   macro avg       0.89      0.87      0.87      6035
weighted avg       0.88      0.88      0.88      6035
 
 Confusion matrix 
 [[3217  173]
 [ 561 2084]]


In [37]:
highest_score_count = 0
highest_score_tfidf = 0
print("Choosing Alpha for Count Vectorizer \n")
for alpha in np.arange(0,1,0.1):
    sub_classifier_count = MultinomialNB(alpha=alpha)
    sub_classifier_count.fit(X_train_count,Y_train_count)
    Y_pred_hyper_count = sub_classifier_count.predict(X_test_count)
    score = accuracy_score(Y_test_count,Y_pred_hyper_count)
    if score > highest_score_count:
        highest_score_count = score
        classfier_count = sub_classifier_count
    print("Alpha : {},Score {}".format(alpha,score))
print("\n Choosing Alpha for Tfidf Vectorizer \n")
for alpha in np.arange(0,1,0.1):
    sub_classifier_tfidf = MultinomialNB(alpha=alpha)
    sub_classifier_tfidf.fit(X_train_tfidf,Y_train_tfidf)
    Y_pred_hyper_tfidf = sub_classifier_tfidf.predict(X_test_tfidf)
    score = accuracy_score(Y_test_tfidf,Y_pred_hyper_tfidf)
    if score > highest_score_tfidf:
        highest_score_tfidf = score
        classfier_tfidf = sub_classifier_tfidf
    print("Alpha : {},Score {}".format(alpha,score))
print("\n\n Best Alpha value for count vectorizer:",classfier_count)
print("\n Best Alpha value for tfidf vectorizer:" ,classfier_tfidf)

Choosing Alpha for Count Vectorizer 

Alpha : 0.0,Score 0.8901408450704226
Alpha : 0.1,Score 0.9042253521126761
Alpha : 0.2,Score 0.9028997514498758
Alpha : 0.30000000000000004,Score 0.9032311516155758
Alpha : 0.4,Score 0.903893951946976
Alpha : 0.5,Score 0.9028997514498758
Alpha : 0.6000000000000001,Score 0.9025683512841757
Alpha : 0.7000000000000001,Score 0.9022369511184756
Alpha : 0.8,Score 0.9024026512013256
Alpha : 0.9,Score 0.9020712510356255

 Choosing Alpha for Tfidf Vectorizer 

Alpha : 0.0,Score 0.8642916321458161
Alpha : 0.1,Score 0.8783761391880696
Alpha : 0.2,Score 0.8793703396851699
Alpha : 0.30000000000000004,Score 0.8810273405136703
Alpha : 0.4,Score 0.8818558409279205
Alpha : 0.5,Score 0.88003314001657
Alpha : 0.6000000000000001,Score 0.8792046396023198
Alpha : 0.7000000000000001,Score 0.8792046396023198
Alpha : 0.8,Score 0.8783761391880696
Alpha : 0.9,Score 0.8783761391880696


 Best Alpha value for count vectorizer: MultinomialNB(alpha=0.1)

 Best Alpha value for tfi

In [39]:
feature_names_count = count.get_feature_names_out()
feature_names_tfidf = tfidf.get_feature_names_out()

In [41]:
# Log probabilities of features per class
log_probs = sub_classifier_count.feature_log_prob_
print(log_probs)


[[ -7.30786336 -11.45451921  -9.28229594 ...  -9.28229594 -11.45451921
   -9.98818215]
 [ -6.79717072  -8.41465697  -9.64248099 ... -10.81255224  -8.52114045
  -10.06533784]]


In [48]:
print("\nFake words from Count Vectorizer\n")
# Log probabilities of features for the "fake" class (assuming class 0 represents "fake")
fake_class_log_probs = sub_classifier_count.feature_log_prob_[0]
# Pair log probabilities with feature names
fake_words_count = sorted(zip(fake_class_log_probs, feature_names_count))[:10]
for word, log_prob in fake_words_count:
    print(f"Word: {word}, Log Probability: {log_prob}")

print("\nFake words from Tfidf Vectorizer\n")
# Log probabilities of features for the "fake" class in Tfidf
fake_class_log_probs_tfidf = sub_classifier_tfidf.feature_log_prob_[0]
fake_words_tfidf = sorted(zip(fake_class_log_probs_tfidf, feature_names_tfidf))[:10]
for word, log_prob in fake_words_tfidf:
    print(f"Word: {word}, Log Probability: {log_prob}")



Fake words from Count Vectorizer

Word: -11.454519214081834, Log Probability: 000 email
Word: -11.454519214081834, Log Probability: 00pm
Word: -11.454519214081834, Log Probability: 00pm water
Word: -11.454519214081834, Log Probability: 00pm water cooler
Word: -11.454519214081834, Log Probability: 01
Word: -11.454519214081834, Log Probability: 100percentfedup
Word: -11.454519214081834, Log Probability: 100percentfedup com
Word: -11.454519214081834, Log Probability: 11 16
Word: -11.454519214081834, Log Probability: 11 2016
Word: -11.454519214081834, Log Probability: 2006

Fake words from Tfidf Vectorizer

Word: -10.258388662446865, Log Probability: 000 email
Word: -10.258388662446865, Log Probability: 00pm
Word: -10.258388662446865, Log Probability: 00pm water
Word: -10.258388662446865, Log Probability: 00pm water cooler
Word: -10.258388662446865, Log Probability: 01
Word: -10.258388662446865, Log Probability: 100percentfedup
Word: -10.258388662446865, Log Probability: 100percentfedup c

In [56]:
# Real News Words
print("\nReal words from Count Vectorizer\n")
# Log probabilities of features for the "real" class (assuming class 1 represents "real")
real_class_log_probs = sub_classifier_count.feature_log_prob_[1]
# Pair log probabilities with feature names
real_words_count = sorted(zip(real_class_log_probs, feature_names_count), reverse=True)[:10]
for log_prob, word in real_words_count:
    print(f"Word: {word}, Log Probability: {log_prob}")

print("\nReal words from Tfidf Vectorizer\n")
# Log probabilities of features for the "real" class in Tfidf
real_class_log_probs_tfidf = sub_classifier_tfidf.feature_log_prob_[1]
real_words_tfidf = sorted(zip(real_class_log_probs_tfidf, feature_names_tfidf), reverse=True)[:10]
for log_prob, word in real_words_tfidf:
    print(f"Word: {word}, Log Probability: {log_prob}")



Real words from Count Vectorizer

Word: trump, Log Probability: -3.991929556794031
Word: hillari, Log Probability: -4.279248014915401
Word: clinton, Log Probability: -4.408426317231802
Word: elect, Log Probability: -4.911437970465705
Word: new, Log Probability: -5.18213727028834
Word: comment, Log Probability: -5.206341759702595
Word: video, Log Probability: -5.304964893722169
Word: war, Log Probability: -5.332376386139956
Word: hillari clinton, Log Probability: -5.374956138909219
Word: us, Log Probability: -5.404384451839828

Real words from Tfidf Vectorizer

Word: hillari, Log Probability: -4.934778691096527
Word: trump, Log Probability: -4.960884523428452
Word: clinton, Log Probability: -5.103351190132023
Word: elect, Log Probability: -5.423377763536257
Word: comment, Log Probability: -5.732489702949988
Word: war, Log Probability: -5.74427338205135
Word: video, Log Probability: -5.750641633945172
Word: us, Log Probability: -5.805652192713543
Word: hillari clinton, Log Probability: 

In [57]:
passive_classifier_count = PassiveAggressiveClassifier(n_iter_no_change = 50)
passive_classifier_count.fit(X_train_count,Y_train_count)
Y_pred_count_passive = passive_classifier_count.predict(X_test_count)
passive_classifier_tfidf = PassiveAggressiveClassifier(n_iter_no_change = 50)
passive_classifier_tfidf.fit(X_train_tfidf,Y_train_tfidf)
Y_pred_tfidf_passive = passive_classifier_count.predict(X_test_tfidf)

In [58]:
acc_count_passive = accuracy_score(Y_test_count,Y_pred_count_passive)
classification_count_passive = classification_report(Y_test_count,Y_pred_count_passive)
confusion_matrix_count_passive = confusion_matrix(Y_test_count,Y_pred_count_passive)
acc_tfidf_passive = accuracy_score(Y_test_tfidf,Y_pred_tfidf_passive)
classification_tfidf_passive = classification_report(Y_test_tfidf,Y_pred_tfidf_passive)
confusion_matrix_tfidf_passive = confusion_matrix(Y_test_tfidf,Y_pred_tfidf_passive)

In [59]:
print("FOR COUNT VECTORIZER - Passive Aggressive Model: \n")
print(" \n Accuracy : ",acc_count_passive,"\n","Classification report \n\n",classification_count_passive,"\n","Confusion matrix \n\n",confusion_matrix_count_passive,"\n\n\n")
print("FOR TFIDF VECTORIZER - Passive Aggressive Model: \n")
print(" \n Accuracy : ",acc_tfidf_passive,"\n","Classification report \n\n",classification_tfidf_passive,"\n","Confusion matrix \n\n",confusion_matrix_tfidf_passive)

FOR COUNT VECTORIZER - Passive Aggressive Model: 

 
 Accuracy :  0.911847555923778 
 Classification report 

               precision    recall  f1-score   support

           0       0.93      0.92      0.92      3390
           1       0.89      0.91      0.90      2645

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035
 
 Confusion matrix 

 [[3108  282]
 [ 250 2395]] 



FOR TFIDF VECTORIZER - Passive Aggressive Model: 

 
 Accuracy :  0.5376967688483845 
 Classification report 

               precision    recall  f1-score   support

           0       0.98      0.18      0.30      3390
           1       0.49      1.00      0.65      2645

    accuracy                           0.54      6035
   macro avg       0.73      0.59      0.48      6035
weighted avg       0.76      0.54      0.46      6035
 
 Confusion matrix 

 [[ 612 2778]
 [  12 2633]]


In [60]:
stopwords.words("english")

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [61]:
wordvec = Word2Vec(words,min_count=1)

In [63]:
vocab = wordvec.wv.key_to_index 

In [64]:
vocab

{'new': 0,
 'time': 1,
 'york': 2,
 'trump': 3,
 'breitbart': 4,
 'hillari': 5,
 'clinton': 6,
 'donald': 7,
 'elect': 8,
 'u': 9,
 'say': 10,
 'obama': 11,
 'presid': 12,
 'state': 13,
 'video': 14,
 'war': 15,
 'vote': 16,
 'russia': 17,
 'report': 18,
 'us': 19,
 'comment': 20,
 'attack': 21,
 'world': 22,
 'year': 23,
 'american': 24,
 'day': 25,
 'news': 26,
 'call': 27,
 'email': 28,
 'fbi': 29,
 'hous': 30,
 'america': 31,
 'get': 32,
 'kill': 33,
 'white': 34,
 'polic': 35,
 'media': 36,
 'campaign': 37,
 'plan': 38,
 'democrat': 39,
 'protest': 40,
 'support': 41,
 'bill': 42,
 'first': 43,
 'one': 44,
 'brief': 45,
 'make': 46,
 'show': 47,
 'win': 48,
 'even': 49,
 'black': 50,
 'syria': 51,
 'anti': 52,
 '2016': 53,
 'back': 54,
 'take': 55,
 'right': 56,
 'watch': 57,
 'nation': 58,
 'russian': 59,
 'man': 60,
 'like': 61,
 '2': 62,
 'peopl': 63,
 'die': 64,
 'go': 65,
 'investig': 66,
 'break': 67,
 'use': 68,
 'may': 69,
 'muslim': 70,
 '3': 71,
 'million': 72,
 'polit':

In [65]:
similar = wordvec.wv.most_similar('hillari')
similar

[('email', 0.9861606359481812),
 ('investig', 0.9815990328788757),
 ('reopen', 0.9812924265861511),
 ('clinton', 0.9795011281967163),
 ('fbi', 0.9763426184654236),
 ('campaign', 0.9634321331977844),
 ('foundat', 0.9577846527099609),
 ('batch', 0.9575839042663574),
 ('wikileak', 0.9568206667900085),
 ('crook', 0.9566701650619507)]

In [66]:
vector = wordvec.wv["hillari"]
vector

array([-1.3813375e-01,  4.9354723e-01, -1.2448050e-01, -4.4216031e-01,
       -1.8740304e-01, -9.5923889e-01,  6.6813254e-01,  7.9852104e-01,
       -2.7554545e-01, -5.4869151e-01, -4.7432937e-02, -7.0848370e-01,
        3.5839838e-01, -2.4487990e-01, -5.1892668e-01, -2.2385050e-01,
        3.2817334e-01, -1.2576865e+00, -3.0118191e-01, -1.4599354e+00,
       -3.1034732e-01,  6.7789531e-01,  1.2991782e+00,  1.7573546e-01,
       -6.9863707e-01,  2.0968570e-01, -4.1994256e-01, -5.2981108e-01,
       -4.6813601e-01, -8.9795463e-02,  9.0834373e-01, -1.3972130e-01,
        2.1713860e-02, -8.1755340e-01, -1.9144523e-01,  1.0889226e+00,
       -2.3727088e-01, -5.8867621e-01, -8.7400007e-01, -1.3348289e+00,
        4.9799395e-01, -7.5352150e-01,  3.6584890e-01, -1.8759598e-01,
        7.8699434e-01, -6.4316458e-01, -4.3550834e-01, -5.8537565e-02,
       -7.9466074e-05,  5.1771706e-01,  2.5657224e-02, -5.8123916e-01,
       -5.4468539e-02,  5.2433968e-01, -9.4870222e-01,  7.9099402e-02,
      

In [67]:
for i,j in zip(words,range(0,len(words))):
    print(j,i)

0 ['hous', 'dem', 'aid', 'even', 'see', 'comey', 'letter', 'jason', 'chaffetz', 'tweet']
1 ['flynn', 'hillari', 'clinton', 'big', 'woman', 'campu', 'breitbart']
2 ['truth', 'might', 'get', 'fire']
3 ['15', 'civilian', 'kill', 'singl', 'us', 'airstrik', 'identifi']
4 ['iranian', 'woman', 'jail', 'fiction', 'unpublish', 'stori', 'woman', 'stone', 'death', 'adulteri']
5 ['jacki', 'mason', 'hollywood', 'would', 'love', 'trump', 'bomb', 'north', 'korea', 'lack', 'tran', 'bathroom', 'exclus', 'video', 'breitbart']
6 ['beno', 'hamon', 'win', 'french', 'socialist', 'parti', 'presidenti', 'nomin', 'new', 'york', 'time']
7 ['back', 'channel', 'plan', 'ukrain', 'russia', 'courtesi', 'trump', 'associ', 'new', 'york', 'time']
8 ['obama', 'organ', 'action', 'partner', 'soro', 'link', 'indivis', 'disrupt', 'trump', 'agenda']
9 ['bbc', 'comedi', 'sketch', 'real', 'housew', 'isi', 'caus', 'outrag']
10 ['russian', 'research', 'discov', 'secret', 'nazi', 'militari', 'base', 'treasur', 'hunter', 'arctic',

In [68]:
words[562]

[]

In [69]:
rows_to_be_removed = []
for i,j in zip(words,range(0,500000)):
    if i == []:
        rows_to_be_removed.append(j)

In [70]:
len(rows_to_be_removed)

51

In [71]:
series= pd.Series(words)

In [72]:
df = pd.DataFrame(series,columns = ["news"])

In [73]:
df

,news
0,"[hous, dem, aid, even, see, comey, letter, jas..."
1,"[flynn, hillari, clinton, big, woman, campu, b..."
2,"[truth, might, get, fire]"
3,"[15, civilian, kill, singl, us, airstrik, iden..."
4,"[iranian, woman, jail, fiction, unpublish, sto..."
...,...
18280,"[rapper, trump, poster, child, white, supremaci]"
18281,"[n, f, l, playoff, schedul, matchup, odd, new,..."
18282,"[maci, said, receiv, takeov, approach, hudson,..."
18283,"[nato, russia, hold, parallel, exercis, balkan]"


In [74]:
df.drop(df.index[rows_to_be_removed], inplace=True)

In [75]:
def document_vector(doc):
    doc = [word for word in doc if word in wordvec.wv.vocab]
    return np.mean(wordvec.wv[doc],axis = 0)

In [77]:
def document_vector(doc):
    # Check if each word is in the model's vocabulary using key_to_index
    doc = [word for word in doc if word in wordvec.wv.key_to_index]
    # Compute the mean vector
    if len(doc) > 0:  # Avoid computing the mean of an empty list
        return np.mean(wordvec.wv[doc], axis=0)
    else:
        return np.zeros(wordvec.vector_size)  # Return a zero vector if no words are in the vocabulary


In [79]:
X_wordvec = pd.DataFrame(df.news.apply(document_vector))
X_wordvec.reset_index(inplace=True,drop =True)

In [80]:
X_wordvec


,news
0,"[-0.27864942, 0.50948167, 0.07816382, 0.121013..."
1,"[-0.23045172, 0.5240415, 0.031126011, -0.01741..."
2,"[-0.31074435, 0.5740769, 0.09682138, 0.1361839..."
3,"[-0.24343921, 0.46229264, 0.07642228, 0.135708..."
4,"[-0.19063859, 0.34762684, 0.053138245, 0.08878..."
...,...
18229,"[-0.2718711, 0.42561105, 0.080355935, 0.028794..."
18230,"[-0.38689947, 0.6240958, 0.076409385, 0.267192..."
18231,"[-0.31447154, 0.5100667, 0.0565509, 0.22422428..."
18232,"[-0.1746446, 0.3266993, 0.061962113, 0.0822757..."


In [81]:
Y_wordvec = Y.drop(index=rows_to_be_removed)
Y_wordvec = Y_wordvec.reset_index(drop =True)

In [82]:
Y_wordvec

0        1
1        0
2        1
3        1
4        1
        ..
18229    0
18230    0
18231    0
18232    1
18233    1
Name: label, Length: 18234, dtype: int64

In [83]:
X_train_wordvec,X_test_wordvec,Y_train_wordvec,Y_test_wordvec = train_test_split(X_wordvec,Y_wordvec,test_size=0.33,random_state = 27)

In [84]:
lr = LogisticRegression()

In [85]:
X_train_wordvec = X_train_wordvec.squeeze('columns')
X_train_wordvec

4138     [-0.16681078, 0.29844114, 0.055102687, 0.08592...
1059     [-0.23650984, 0.40836325, 0.06740679, 0.069949...
11557    [-0.14538755, 0.34287888, 0.023441713, 0.02143...
1664     [-0.2928095, 0.53690165, 0.082635045, 0.181441...
14433    [-0.39223656, 0.64306766, 0.08342942, 0.219515...
                               ...                        
15897    [-0.3623699, 0.5867174, 0.069971114, 0.2533279...
4848     [-0.21465278, 0.38826057, 0.056927484, 0.10344...
14879    [-0.1781547, 0.31485987, 0.05160181, 0.0791548...
3912     [-0.42161933, 0.70331734, 0.0902298, 0.3035249...
5139     [-0.1834448, 0.3850591, 0.04503515, 0.04577931...
Name: news, Length: 12216, dtype: object

In [86]:
X_test_wordvec = X_test_wordvec.squeeze('columns')
X_test_wordvec

15801    [-0.40625653, 0.6846321, 0.08285188, 0.2802051...
3345     [-0.4252449, 0.69728166, 0.08524608, 0.2910628...
7181     [-0.18385923, 0.33567104, 0.0553809, 0.0766808...
5066     [-0.20336191, 0.37854, 0.051365703, 0.08686819...
16842    [-0.17109334, 0.32206893, 0.041952603, 0.07604...
                               ...                        
11148    [-0.24947996, 0.4412125, 0.068963565, 0.135940...
8915     [-0.4037471, 0.66663164, 0.08594853, 0.2788979...
7258     [-0.40656322, 0.702509, 0.122643396, 0.1511068...
2594     [-0.20655322, 0.35289395, 0.06524384, 0.067697...
15554    [-0.3543208, 0.5639231, 0.065234475, 0.2514750...
Name: news, Length: 6018, dtype: object

In [87]:
lr.fit(list(X_train_wordvec),Y_train_wordvec)

LogisticRegression()

In [88]:
Y_pred_wordvec_log = lr.predict(list(X_test_wordvec))

In [89]:
accuracy_score(Y_test_wordvec,Y_pred_wordvec_log)

0.8017613825191093

In [90]:
classfier_wordvec = GaussianNB()

In [91]:
classfier_wordvec.fit(list(X_train_wordvec),Y_train_wordvec)
Y_pred_wordvec_multi = classfier_wordvec.predict(list(X_test_wordvec))

In [92]:
accuracy_score(Y_test_wordvec,Y_pred_wordvec_multi)

0.7522432701894317

In [93]:
df_test

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...


In [94]:
ps = PorterStemmer()
corpus_test = []
for i in range(0,len(df_test)):
    review = re.sub('[^a-zA-Z0-9]',' ',str(df_test['title'][i]))
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    statements = ' '.join(review)
    corpus_test.append(statements)
    words.append(review)

In [95]:
df_test.isna().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [96]:
count_test = CountVectorizer(max_features=5000,ngram_range=(1,3))
X_test_count = count.fit_transform(corpus_test).toarray()

In [97]:
Y_pred_count_passive = passive_classifier_count.predict(X_test_count)

In [98]:
submission= pd.DataFrame({"id":df_test.id,"label":Y_pred_count_passive})
submission.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,0
4,20804,1


In [99]:
filename = "submission.csv"
submission.to_csv(filename, index=False)

## Keras Word Embeddings

In [100]:
voc_size = 5000
sent_length=20
embedding_vector_features=40

In [101]:
messages=X.copy()

In [102]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [103]:
messages.reset_index(inplace=True)

In [104]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2738, 2068, 3428, 3500, 2131, 4394, 1930, 3961, 4376, 2508],
 [1223, 3723, 3886, 71, 3079, 4724, 1865],
 [4722, 134, 4582, 46],
 [879, 756, 2759, 962, 3057, 1679, 4068],
 [3650, 3079, 3182, 844, 2143, 1198, 3079, 2122, 1816, 2083],
 [2118,
  4263,
  526,
  4027,
  3024,
  3614,
  3142,
  1430,
  2163,
  1389,
  1008,
  928,
  3168,
  1938,
  1865],
 [2577, 2172, 2445, 4187, 3737, 4709, 45, 2687, 838, 1229, 3112],
 [4212, 263, 3737, 4691, 3200, 4700, 3614, 2271, 838, 1229, 3112],
 [434, 1080, 1060, 4714, 2881, 1887, 4001, 4803, 3614, 1217],
 [3230, 4945, 4576, 2605, 3578, 974, 3255, 1593],
 [1129, 2384, 2124, 3487, 324, 1215, 1285, 1077, 3812, 6, 4329],
 [3057, 701, 2131, 1887, 3614, 3200],
 [4973, 137, 2532, 2219, 1445, 4320, 44, 4995, 2607],
 [2370, 619, 2498, 3276, 1829, 1996, 3739, 838, 1229, 3112],
 [3605, 870, 3461, 654, 3205, 838, 1229, 3112],
 [2740, 3817, 2127, 1763, 4799, 3791, 3526, 1940, 3122, 1550, 2040, 1800],
 [1032, 4145, 3723],
 [2019, 4087, 4837, 2322, 3614, 3804, 42

In [105]:
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2738 2068 3428 ...    0    0    0]
 [1223 3723 3886 ...    0    0    0]
 [4722  134 4582 ...    0    0    0]
 ...
 [ 410  486 4314 ...    0    0    0]
 [2227 3200 2688 ...    0    0    0]
 [2640  391 1625 ...    0    0    0]]


In [106]:
embedded_docs[0]

array([2738, 2068, 3428, 3500, 2131, 4394, 1930, 3961, 4376, 2508,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [107]:
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [108]:
X_final=np.array(embedded_docs)
y_final=np.array(Y)

In [109]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [110]:
X_train_embed, X_test_embed, Y_train_embed, Y_test_embed = train_test_split(X_final, y_final, test_size=0.33, random_state=27)

In [111]:
model.fit(X_train_embed,Y_train_embed,validation_data=(X_test_embed,Y_test_embed),epochs=20,batch_size=64)

Epoch 1/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.7567 - loss: 0.4408 - val_accuracy: 0.9153 - val_loss: 0.2206
Epoch 2/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9485 - loss: 0.1426 - val_accuracy: 0.9251 - val_loss: 0.1987
Epoch 3/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9671 - loss: 0.0924 - val_accuracy: 0.9196 - val_loss: 0.2276
Epoch 4/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9709 - loss: 0.0775 - val_accuracy: 0.9191 - val_loss: 0.2403
Epoch 5/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9840 - loss: 0.0506 - val_accuracy: 0.9171 - val_loss: 0.2837
Epoch 6/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9854 - loss: 0.0417 - val_accuracy: 0.9112 - val_loss: 0.3155
Epoch 7/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9891 - loss: 0.0342 - val_accuracy: 0.9109 - val_loss: 0.3252
Epoch 8/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9912 - loss: 0.0298 - val_accu

In [119]:
predict_x=model.predict(X_test_embed) 
Y_pred_embed=np.argmax(predict_x,axis=1)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [120]:
accuracy_score(Y_test_embed,Y_pred_embed)

0.5617232808616405